In [1]:
def get_creators(record: dict) -> list:
    match record:
        case {"type": "book", "authors": [*names]}:
            return names
        case {"type": "book", "author": name}:
            return [name]
        # Capturing extra arguments
        case {"type": "series", "author": name, **extra}:
            return [name, extra]
        case {"type": "book"}:
            raise ValueError(f"Invalid 'book' record: {record}")
        case _:
            raise ValueError(f"Invalid record: {record!r}")


In [6]:
# !r calls the repr in the f-string
record = {"type": "book", "authors": [*range(10)]}
print(f"{record!r}\n{record}")
print(get_creators(record))
get_creators({"type": "book"})


{'type': 'book', 'authors': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
{'type': 'book', 'authors': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


ValueError: Invalid 'book' record: {'type': 'book'}

In [ ]:
# Mapping patterns succeed on partial matches!!!
# Se the title field
get_creators({"type": "book", "author": "Neil Gaiman", "title": "Sandman"})


['Neil Gaiman']

In [ ]:
# Mapping patterns succeed on partial matches!!!
# Se the title field
get_creators(
    {
        "type": "series",
        "author": "Neil Gaiman",
        "title": "Sandman",
        "episodies": 11,
        "chapters": 1,
    }
)


['Neil Gaiman', {'title': 'Sandman', 'episodies': 11, 'chapters': 1}]

In [ ]:
def sponge(**bob):
    return bob


sponge(**{"x": 10, "y": 20}, z="3", **{"k": "2"})


{'x': 10, 'y': 20, 'z': '3', 'k': '2'}

In [ ]:
d1 = {"a": 0, **{"x": 1, "y": 2}, "z": 3, **{"z": 4, "x": 2}}
d1

{'a': 0, 'x': 2, 'y': 2, 'z': 4}

In [ ]:
d2 = {'a':2, 'b':4, 'c': 6}
d1 | d2

{'a': 2, 'x': 2, 'y': 2, 'z': 4, 'b': 4, 'c': 6}

In [ ]:
d1 |= d2
d1

{'a': 2, 'x': 2, 'y': 2, 'z': 4, 'b': 4, 'c': 6}

In [ ]:
from collections import abc

In [ ]:
isinstance(dict(), abc.Mapping)

True

In [ ]:
isinstance(dict(), abc.MutableMapping)

True

In [11]:
from collections import UserDict
# To implement a custom mapping, it's easier to extend collections.UserDict
# ot to wrap a dict by composition instead of subclassing these
# ABCs (ABstract Class).
print(isinstance(dict(), UserDict))
class MyDict(UserDict):
    pass

# Even if you implement your dict, the keys need to be hashable
print(isinstance(MyDict(), UserDict))

print(isinstance(MyDict(), dict))


False
True
False


In [ ]:
# Tupjes are hashable only if all its itens are hashable
print(hash((1000000,)))
# List are unhashable
hash([1000000,])

543870488427927284


TypeError: unhashable type: 'list'

In [12]:
from collections import defaultdict, OrderedDict

In [19]:
words = ['a', 'b', 'c', 'd', 'e']
index = defaultdict(list)
for w in words:
    index[w].append(w)
index['k'] # Gera uma lista vazia
index

defaultdict(list,
            {'a': ['a'],
             'b': ['b'],
             'c': ['c'],
             'd': ['d'],
             'e': ['e'],
             'k': []})

In [ ]:
for w in sorted(index, key=str.upper):
    print(w, index[w])


a ['a']
b ['b']
c ['c']
d ['d']
e ['e']


In [ ]:
print(index.default_factory('g')) # Não adiciona o item ao dicionário, mas 
print(index.__missing__('k'))
index

['g']
[]


defaultdict(list,
            {'a': ['a'],
             'b': ['b'],
             'c': ['c'],
             'd': ['d'],
             'e': ['e'],
             'k': []})

In [30]:
# default dict é equivalente
index = defaultdict(list)
index['k'].append(0)
index['k'].append(1)
print(index)

# Equivalente
index = dict()
index['k'] = list()
index['k'].append(0)
index['k'].append(1)
print(index)


defaultdict(<class 'list'>, {'k': [0, 1]})
{'k': [0, 1]}


In [31]:
class MyDict(UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default

    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

my_dict = MyDict()
my_dict['1'] = 1
my_dict[1]

1

In [37]:
# Variations of dict
from collections import OrderedDict, ChainMap, Counter
from shelve import Shelf

In [35]:
ordered_dict = OrderedDict({n:chr(n) for n in range(65,65+5)})
ordered_dict.move_to_end(65)
ordered_dict.move_to_end(66)
ordered_dict.move_to_end(67)

print(ordered_dict)
for o in ordered_dict.items():
    print(o)

OrderedDict([(68, 'D'), (69, 'E'), (65, 'A'), (66, 'B'), (67, 'C')])
(68, 'D')
(69, 'E')
(65, 'A')
(66, 'B')
(67, 'C')


In [ ]:
# ChainMap combine mappings and allow to seach them as one
# The ChainMap holds references to them
d1 = {'a':1, 'b':2}
d2 = {'a':'a', 'c':'c'}
chain = ChainMap(d1, d2)
print(chain)

chain['a'], chain['c']

ChainMap({'a': 1, 'b': 2}, {'a': 'a', 'c': 'c'})


(1, 'c')

In [ ]:
chain['a'] = 2
d1

{'a': 2, 'b': 2}

In [38]:
ct = Counter("abrakadabra")
print(ct, "\n", ct["a"])
ct.update("alakazam")
print(ct, "\n", ct["l"])
ct.update("zzzzzzzzz")
print(ct.most_common(4))
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'k': 1, 'd': 1}) 
 5
Counter({'a': 9, 'b': 2, 'r': 2, 'k': 2, 'd': 1, 'l': 1, 'z': 1, 'm': 1}) 
 1
[('z', 10), ('a', 9), ('b', 2), ('r', 2)]


Counter({'a': 9, 'b': 2, 'r': 2, 'k': 2, 'd': 1, 'l': 1, 'z': 10, 'm': 1})

In [ ]:
d3 = {n:chr(n) for n in range(65,70)}
# items is super util
d3.values(), d3.keys(), d3.items()

(dict_values(['A', 'B', 'C', 'D', 'E']),
 dict_keys([65, 66, 67, 68, 69]),
 dict_items([(65, 'A'), (66, 'B'), (67, 'C'), (68, 'D'), (69, 'E')]))

In [39]:
class StrKeyDict(UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

strkey = StrKeyDict()
strkey[1] = 1
strkey['2'] = 2
strkey['1'], strkey[2]

(1, 2)

In [61]:
from types import MappingProxyType
d = defaultdict(list)
d[1] = 1
d_proxy = MappingProxyType(d)
d[2] = 2
d_proxy[3]
try:
    d_proxy[4] = 2
except Exception as e:
    print(e)
d_proxy

'mappingproxy' object does not support item assignment


mappingproxy({1: 1, 2: 2, 3: []})

In [90]:
set_a = {'a','b','c','d'}
set_b = frozenset(('c','d','e','f'))

print(set_a, set_b)
print(set_a|set_b)
print(set_a&set_b)
print(set_a^set_b)
print(set_a-set_b)
print(set_b-set_a)

{'a', 'c', 'd', 'b'} frozenset({'e', 'c', 'f', 'd'})
{'f', 'b', 'c', 'a', 'd', 'e'}
{'c', 'd'}
{'f', 'a', 'b', 'e'}
{'a', 'b'}
frozenset({'e', 'f'})


In [98]:
import numpy as np

In [344]:
c = Counter()
for k in range(10000000):
    # Reinicia o set e a lista
    S = set()
    L = []
    # Loop até o set ser reordenado internamente
    for i in range(100000):
        # Adicionar maior aleatoriedade nos hashs
        I = i+np.random.normal(loc=1, scale=0.05)
        if I not in S:
            S.add(I)
            L.append(I)
            # O set foi reordenado
            if L != list(S):
                c.update([i])
                break
                

In [346]:
c

Counter({3: 1643172,
         76: 1856500,
         2: 2501271,
         306: 3078566,
         4: 588018,
         1228: 332473})